In [1]:
#!pip install gmaps
#!pip install -U jupyter
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#!jupyter nbextension enable --py gmaps
#!pip install -e 

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 
# Import API key
from api_keys import g_key

In [3]:
#import csv and read into cs file
city_csv = "../WeatherPy/city_info.csv"
city_df = pd.read_csv(city_csv, encoding="utf-8")


In [4]:
#Designate key to be used
gmaps.configure(api_key=g_key)
#Set variables for heat map
locations = city_df[["City Latitude", "City Longitude"]]
humidity = city_df["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Creating ideal temperature
ideal_temp_df = city_df.loc [(city_df["Max Temperature"] <=80)&(city_df["Humidity"] <=30)&(city_df["Cloud Coverage"] <=0)]
#Showing data frame
ideal_temp_df

,Cities,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City datetime
16,mount isa,-20.73,139.50,78.80,27,0,8.05,AU,1595564881
23,kruisfontein,-34.00,24.73,62.92,27,0,11.10,ZA,1595564791
130,alice springs,-23.70,133.88,75.00,19,0,6.93,AU,1595565318
189,luderitz,-26.65,15.16,69.80,19,0,3.36,NaN,1595564812
396,oranjemund,-28.55,16.43,66.49,26,0,12.06,NaN,1595565319
399,yulara,-25.24,130.99,69.80,24,0,9.17,AU,1595565452
443,javanrud,34.80,46.52,73.40,27,0,3.47,IR,1595565331
478,calama,-22.47,-68.93,35.60,24,0,3.36,CL,1595565469
548,leh,34.17,77.58,66.42,22,0,3.20,IN,1595565484


In [6]:
#Create copy of ideal_temp_df
hotel_df = ideal_temp_df.copy()
#Create empty column for "Hotel Name"
hotel_df["Hotel Name"]=" "
#Setting variables for params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_type = "lodging"
radius = 5000
#For loop to go find first hotel in 5000 meter range from the given lat,lng in the data frame
for index, row in ideal_temp_df.iterrows():
        lat_loc = row["City Latitude"]
        lng_loc = row["City Longitude"]
        location = f'{lat_loc}, {lng_loc}'
        params = {
        "location": location,
        "types": target_type,
        "radius": radius,
        "key": g_key
        }
        response = requests.get(base_url, params).json()
        #Try/except loop so code the doesn't crash
        try:
            hotel_df.loc[index,"Hotel Name"]=response["results"][0]["name"]
        except:
            pass
#Showing data frame
hotel_df

,Cities,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City datetime,Hotel Name
16,mount isa,-20.73,139.50,78.80,27,0,8.05,AU,1595564881,ibis Styles Mt Isa Verona
23,kruisfontein,-34.00,24.73,62.92,27,0,11.10,ZA,1595564791,Oyster Bay House Rental
130,alice springs,-23.70,133.88,75.00,19,0,6.93,AU,1595565318,Desert Palms Alice Springs
189,luderitz,-26.65,15.16,69.80,19,0,3.36,NaN,1595564812,LÜDERITZ NEST HOTEL
396,oranjemund,-28.55,16.43,66.49,26,0,12.06,NaN,1595565319,Fisherman's Cove
399,yulara,-25.24,130.99,69.80,24,0,9.17,AU,1595565452,Desert Gardens Hotel - Ayers Rock Resort
443,javanrud,34.80,46.52,73.40,27,0,3.47,IR,1595565331,هتل چیا
478,calama,-22.47,-68.93,35.60,24,0,3.36,CL,1595565469,Park Hotel Calama
548,leh,34.17,77.58,66.42,22,0,3.20,IN,1595565484,Hotel Ladakh Greens


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["City Latitude", "City Longitude"]]

In [8]:
#Create hotel layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content= hotel_info
)
#Plot hotel layer
fig = gmaps.figure()
#Add layer
fig.add_layer(hotel_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
#Plot layers
fig = gmaps.figure()
#Add layers
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
#Display figures
fig

Figure(layout=FigureLayout(height='420px'))